In [ ]:
# %load nbheader.py
%reload_ext autoreload
%autoreload 2

from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col as S
from pyspark.sql import DataFrame, Row, Window
import os
import sys
import json
import datetime
import re
import pandas as pd
import numpy as np

In [20]:
spark = SparkSession.builder.master("local[4]").getOrCreate()
spark.getActiveSession()
# spark.stop()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 12:45:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54366)
Traceback (most recent call last):
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/pmolnar/.b

In [67]:
quant = np.random.randint(0, 1000, (20))
delta = np.random.randint(-50, 50, (20))
k = (delta*delta).argmin() # find smallest error
print(k, quant[k], delta[k])
quant[k] = 0
print(quant)
print(delta)

17 670 -1
[ 26  83 686 438 205 477 885 183 729 827 331 718 978 592 183 399 448   0
 908  70]
[-41 -30  34  17 -46  -8  29 -22  -6  34 -40  35 -31  26 -34   3   4  -1
  -5  35]


In [68]:
data = [
    Row(sku=j+1, quant=int(q), delta=int(e)) for j, (q, e) in enumerate(zip(quant, delta))
]

In [69]:
schema=T.StructType([
    T.StructField('sku', T.IntegerType(), False),
    T.StructField('quant', T.IntegerType(), False),
    T.StructField('delta', T.IntegerType(), False)
])

In [70]:
df = spark.createDataFrame(data, schema=schema)
df.printSchema()
df.toPandas().set_index('sku').T

root
 |-- sku: integer (nullable = false)
 |-- quant: integer (nullable = false)
 |-- delta: integer (nullable = false)



sku,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
quant,26,83,686,438,205,477,885,183,729,827,331,718,978,592,183,399,448,0,908,70
delta,-41,-30,34,17,-46,-8,29,-22,-6,34,-40,35,-31,26,-34,3,4,-1,-5,35


**Step 1:**
Compute Absolute Percentage Error as $\frac{\delta}{quant}$

In [97]:
df2 = df.withColumn("ape", F.abs('delta')/S('quant'))

**Step 2:** Review results

In [95]:
df2.head(3)

[Row(sku=1, quant=26, delta=-41, ape=1.5769230769230769),
 Row(sku=2, quant=83, delta=-30, ape=0.3614457831325301),
 Row(sku=3, quant=686, delta=34, ape=0.04956268221574344)]

**Step 3:** Calculate mean value

In [98]:
MAPE = df2.agg(F.mean("ape")).collect()[0][0]
print(f"Mean Absolute Percentage Error (MAPE): {MAPE:.4}")

25/09/19 14:13:16 ERROR Executor: Exception in task 3.0 in stage 54.0 (TID 126)
org.apache.spark.SparkArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 1 in cell [97]

	at org.apache.spark.sql.errors.QueryExecutionErrors$.divideByZeroError(QueryExecutionErrors.scala:203)
	at org.apache.spark.sql.errors.QueryExecutionErrors.divideByZeroError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegen

ArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 1 in cell [97]


In [73]:
df2.orderBy(S('quant')).head(3)

25/09/19 13:08:42 ERROR Executor: Exception in task 3.0 in stage 22.0 (TID 67)
org.apache.spark.SparkArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 1 in cell [72]

	at org.apache.spark.sql.errors.QueryExecutionErrors$.divideByZeroError(QueryExecutionErrors.scala:203)
	at org.apache.spark.sql.errors.QueryExecutionErrors.divideByZeroError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:50)
	at scala.collection.Iterator$$a

ArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 1 in cell [72]


In [92]:
MAPE = df2.filter(S('quant')>0).agg(F.mean("ape")).collect()[0][0]
print(f"Mean Absolute Error (MAPE): {MAPE:.4}")

Mean Absolute Error (MAPE): 0.1802
